In [1]:
pip install shap

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
from tqdm import tqdm
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
import shap

df = pd.read_csv('/home/maia-user/myl/sc/Datasets/processed_total.csv')
df.head()

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,0.750561,1.416667e-07,0.000005,0.0,9.302326e-07,0.0,0.000242,0.002581,0.00101,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
1,0.750561,1.166667e-07,0.000005,0.0,9.302326e-07,0.0,0.000242,0.002581,0.00101,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
2,0.750561,1.166667e-07,0.000005,0.0,9.302326e-07,0.0,0.000242,0.002581,0.00101,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
3,0.750561,1.166667e-07,0.000005,0.0,9.302326e-07,0.0,0.000242,0.002581,0.00101,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
4,0.755108,1.333333e-07,0.000005,0.0,9.302326e-07,0.0,0.000242,0.002581,0.00101,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN


In [3]:
print(df.shape)
print(df["Label"].value_counts())

(2827876, 79)
Label
BENIGN                        2271320
DoS Hulk                       230124
PortScan                       158804
DDoS                           128025
DoS GoldenEye                   10293
FTP-Patator                      7935
SSH-Patator                      5897
DoS slowloris                    5796
DoS Slowhttptest                 5499
Bot                              1956
Web Attack � Brute Force         1507
Web Attack � XSS                  652
Infiltration                       36
Web Attack � Sql Injection         21
Heartbleed                         11
Name: count, dtype: int64


In [10]:

# 假设你已经加载好 df，并完成归一化，包含 "Label" 列
rare_classes = ["Infiltration", "Web Attack � XSS", "Web Attack � Sql Injection", "Heartbleed"]
# 固定 BENIGN 样本数（防止混淆 GAN 效果）
benign_df = df[df["Label"] == "BENIGN"].sample(n=1000000, random_state=42)

In [11]:
rare_raw = df[df["Label"].isin(rare_classes)]
df_raw = pd.concat([benign_df, rare_raw], ignore_index=True)

In [12]:
from cgan_module import cgan_augment  

df_single = cgan_augment(df, rare_classes, generate_per_class=500)
rare_single = df_single[df_single["Label"].isin(rare_classes)]
df_single_full = pd.concat([benign_df, rare_single], ignore_index=True)



🔍 正在增强类别: Infiltration
[0/500] Loss_D: 1.3866, Loss_G: 0.7169
[100/500] Loss_D: 1.0270, Loss_G: 1.1372
[200/500] Loss_D: 0.5104, Loss_G: 2.6851
[300/500] Loss_D: 0.2539, Loss_G: 3.0577
[400/500] Loss_D: 1.0448, Loss_G: 1.5087
✅ 生成完成: Infiltration → 500 条

🔍 正在增强类别: Web Attack � XSS
[0/500] Loss_D: 1.3798, Loss_G: 0.6786
[100/500] Loss_D: 0.9164, Loss_G: 1.0851
[200/500] Loss_D: 0.2578, Loss_G: 2.8282
[300/500] Loss_D: 0.6735, Loss_G: 1.2421
[400/500] Loss_D: 0.8237, Loss_G: 1.2880
✅ 生成完成: Web Attack � XSS → 500 条

🔍 正在增强类别: Web Attack � Sql Injection
[0/500] Loss_D: 1.3857, Loss_G: 0.6686
[100/500] Loss_D: 0.8239, Loss_G: 1.4977
[200/500] Loss_D: 0.2121, Loss_G: 3.0252
[300/500] Loss_D: 0.4664, Loss_G: 1.6204
[400/500] Loss_D: 0.8565, Loss_G: 1.3481
✅ 生成完成: Web Attack � Sql Injection → 500 条

🔍 正在增强类别: Heartbleed
[0/500] Loss_D: 1.3943, Loss_G: 0.7158
[100/500] Loss_D: 0.7666, Loss_G: 1.4484
[200/500] Loss_D: 0.1719, Loss_G: 3.3093
[300/500] Loss_D: 0.3327, Loss_G: 2.4138
[400/500] Lo

In [13]:
from evaluate import  evaluate_pipeline
from evaluate import evaluate_on_original_rare_samples

clf_base, X_base, y_base, rare_base_test = evaluate_pipeline(
    df_raw, "A. 无增强 Baseline",
    rare_classes=rare_classes,
    rare_raw_df=rare_raw
)

clf_enh, X_enh, y_enh, rare_enh_test = evaluate_pipeline(
    df_single_full, "B. 单类增强",
    rare_classes=rare_classes,
    rare_raw_df=rare_raw
)


📊 分类报告 (A. 无增强 Baseline):


/home/maia-user/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/maia-user/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/maia-user/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


                            precision    recall  f1-score   support

                    BENIGN     0.9999    1.0000    0.9999    200000
                Heartbleed     1.0000    0.5000    0.6667         2
              Infiltration     1.0000    0.4286    0.6000         7
Web Attack � Sql Injection     0.0000    0.0000    0.0000         4
          Web Attack � XSS     0.9915    0.8931    0.9398       131

                  accuracy                         0.9999    200144
                 macro avg     0.7983    0.5643    0.6413    200144
              weighted avg     0.9999    0.9999    0.9999    200144


📌 当前测试集中的 Rare Attack 分类情况：
                     Class  Test Samples  Correctly Predicted  Recall
              Infiltration             7                    3  0.4286
          Web Attack � XSS           131                  117  0.8931
Web Attack � Sql Injection             4                    0  0.0000
                Heartbleed             2                    1  0.5000

📊 分类报

In [14]:
# 评估未增强模型对原始攻击识别情况
rare_detect_base = evaluate_on_original_rare_samples(clf_base, rare_raw)

# 评估增强后模型对原始攻击识别情况
rare_detect_enh = evaluate_on_original_rare_samples(clf_enh, rare_raw)

# 合并结果对比（可选）
compare = rare_detect_base.copy()
compare = compare.rename(columns={
    "Detected Correctly": "Correct_Before",
    "Recall": "Recall_Before"
})
compare["Correct_After"] = rare_detect_enh["Detected Correctly"]
compare["Recall_After"] = rare_detect_enh["Recall"]

print(compare.to_string(index=False))


                     Class  Original Sample Count  Correct_Before  Recall_Before  Correct_After  Recall_After
              Infiltration                     36              32         0.8889             32        0.8889
          Web Attack � XSS                    652             638         0.9785            641        0.9831
Web Attack � Sql Injection                     21              17         0.8095             19        0.9048
                Heartbleed                     11              10         0.9091             11        1.0000


In [ ]:
import shap
import matplotlib.pyplot as plt
import numpy as np

# ✅ 增强前：Baseline 模型 SHAP 分析
print("🔍 SHAP 分析：Baseline 模型")

explainer_base = shap.Explainer(clf_base, X_base)
shap_values_base = explainer_base(X_base, check_additivity=False)

# 全局特征重要性图（增强前）
shap.summary_plot(shap_values_base, X_base, max_display=15, show=True)

🔍 SHAP 分析：Baseline 模型


  2%|                   | 23358/1000720 [01:56<80:53]       

In [ ]:

# ✅ 增强后：Enhanced 模型 SHAP 分析
print("\n🔍 SHAP 分析：增强后模型")

explainer_enh = shap.Explainer(clf_enh, X_enh)
shap_values_enh = explainer_enh(X_enh, check_additivity=False)

# 全局特征重要性图（增强后）
shap.summary_plot(shap_values_enh, X_enh, max_display=15, show=True)

In [ ]:
print(clf_base.classes_)  # 或 clf_enh.classes_
print(clf_enh.classes_)

In [ ]:
from visualization import plot_tsne_distribution

for a in rare_classes:
    # 按照类别筛选对应的真实样本和 GAN 样本
    real_a = rare_raw[rare_raw["Label"] == a]
    fake_a = rare_single[rare_single["Label"] == a]

    if len(real_a) < 5 or len(fake_a) < 5:
        print(f"⚠️ 样本过少，跳过类别: {a}")
        continue

    # 画出该类别的 t-SNE 图
    plot_tsne_distribution(
        benign_df=benign_df,
        rare_raw=real_a,
        rare_single=fake_a,
        label=a,
        sample_size=50  # 可以调成较小值，避免少样本画不出图
    )



In [ ]:
# 所有 rare real 样本
rare_all_real = rare_raw.copy()

# 所有 GAN 生成的样本
rare_all_fake = rare_single.copy()

# Benign 样本随机采样
benign_sample = benign_df.sample(n=500, random_state=42)

# 打标签
rare_all_real["Type"] = "Real"
rare_all_fake["Type"] = "GAN"
benign_sample["Type"] = "Benign"

# 合并数据
tsne_df = pd.concat([rare_all_real, rare_all_fake, benign_sample], ignore_index=True)

# 选择特征列
feature_cols = [col for col in tsne_df.columns if col not in ["Label", "Type"]]

# 降维 + 可视化
from sklearn.manifold import TSNE
import seaborn as sns
import matplotlib.pyplot as plt

X_tsne = TSNE(n_components=2, perplexity=30, random_state=42).fit_transform(tsne_df[feature_cols])
df_vis = pd.DataFrame(X_tsne, columns=["Dim1", "Dim2"])
df_vis["Type"] = tsne_df["Type"]

plt.figure(figsize=(10, 6))
sns.scatterplot(data=df_vis, x="Dim1", y="Dim2", hue="Type", style=tsne_df["Label"], s=40, alpha=0.7)
plt.title("t-SNE of All Rare Attacks (Real vs GAN vs Benign)")
plt.grid(True)
plt.tight_layout()
plt.show()
